In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score,RocCurveDisplay,auc
from sklearn.model_selection import train_test_split, GridSearchCV,  LeavePGroupsOut, cross_validate, GroupKFold, StratifiedGroupKFold
from sklearn.impute import KNNImputer
from xgboost import XGBClassifier,XGBRFClassifier
import matplotlib.pyplot as plt
import joblib
import random
import itertools
from scipy.stats import randint,uniform


from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV, HalvingGridSearchCV

In [ ]:
#--------------------------------------------------------------------------------------
#------ Data Preprocessing
#--------------------------------------------------------------------------------------
df=pd.read_csv("../data/51061fe4eb0cb9da61ebd13d39879eefb5bfe33f103d3a80c7123deb5e1cd8c9.dat", compression="gzip",sep=";")
#Data Preprocessing
headers = ["target__y","group__uid"]
for c in df.columns:
    if c[:5]=="feats":
        headers.append(c)
pdf = pd.DataFrame(df,columns=headers)
exclude = ["target__y","group__uid","feats__cirk_vikt","feats__bw","feats__sex","feats__pnage_days"]
for column in pdf.columns.values:
    if column not in exclude:
        pdf = pdf[(pdf[column]!=-99999) & (pdf[column] != -999)]
le = LabelEncoder()
le.fit(pdf["group__uid"])
pdf["group__uid"] = le.transform(pdf["group__uid"])
scaler = StandardScaler()
to_scale = [c for c in headers if c not in ["target__y","group__uid"]]
pdf[to_scale] = scaler.fit_transform(pdf[to_scale])
print(f"Missing {pdf.isna().any(axis=1).sum()} out of {pdf.any(axis=1).sum()}")
imputer = KNNImputer()
imputer.fit(pdf)
pdf = pd.DataFrame(imputer.transform(pdf),columns=pdf.columns.values)
print(f"Missing {pdf.isna().any(axis=1).sum()} out of {pdf.any(axis=1).sum()}")
ids = pdf.pop("group__uid")
y = pdf.pop("target__y")
X = pdf
# print(ids)
# print(noPos,noNeg,scale_pos_weight)
#StratifiedGroupKFold
noPos = (y==1).sum()
noNeg = (y==0).sum()
scale_pos_weight=noNeg/noPos
# print(noPos,noNeg,scale_pos_weight)
#--------------------------------------------------------
#--------------------------------------------------------

In [ ]:
#--------------------------------------------------------------------------------------
#------ Validation of of a single combination in the hyperparameter space
#--------------------------------------------------------------------------------------

Xc=np.asarray(X)
yc=np.asarray(y)
rocaucs = []
tr_rocaucs = []
tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)
fig, ax = plt.subplots()

# Test specific id allocation
nx=100
idnx = [i for i,a in enumerate(ids) if a == nx]
sgkf = StratifiedGroupKFold(n_splits = 5)
for i,(train, test) in enumerate(sgkf.split(Xc, yc, groups=ids)):
    X_train = Xc[train]
    X_test = Xc[test]
    y_train = yc[train]
    y_test = yc[test]
    print(f"id {nx} in train {sum([i in train for i in idnx])} (len {len(y_train)}, pos {len([i for i in y_train if i==1])}), in test {sum([i in test for i in idnx])} (len {len(y_test)}, pos {len([i for i in y_test if i==1])})")
    model = XGBClassifier(use_label_encoder=False,
                          eval_metric="auc",
                          objective='binary:logistic',
                          #tree_method='gpu_hist',
                          #gpu_id=0,
                          #predictor='gpu_predictor',
                          scale_pos_weight = scale_pos_weight,
                          n_estimators = 320,
                          learning_rate = 0.34,
                          max_depth = 1,
                          gamma = 0.01,
                          min_child_weight = 0.86,
                          subsample = 0.64,
                          colsample_bytree = 0.92,
                          n_jobs=20,
                          alpha=0.01,
                          reg_lambda = 1.23
                         )

    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_test)[:,1]
    y_tr_pred = model.predict_proba(X_train)[:,1]
    rocaucs.append(roc_auc_score(y_test,y_pred))
    tr_rocaucs.append(roc_auc_score(y_train,y_tr_pred))
    viz = RocCurveDisplay.from_predictions(
        y_test,
        y_pred,
        name="ROC fold {}".format(i),
        alpha=0.3,
        lw=1,
        ax=ax,
    )
    interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)
print(f"avg rocaucs ={np.mean(rocaucs)}, stde ={np.std(rocaucs, ddof=1) / np.sqrt(np.size(rocaucs))}")
print(f"avg training rocaucs ={np.mean(tr_rocaucs)}, stde ={np.std(tr_rocaucs, ddof=1) / np.sqrt(np.size(tr_rocaucs))}")

ax.plot([0, 1], [0, 1], linestyle="--", lw=2, color="r", label="Chance", alpha=0.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(
    mean_fpr,
    mean_tpr,
    color="b",
    label=r"Mean ROC (AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
    lw=2,
    alpha=0.8,
)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(
    mean_fpr,
    tprs_lower,
    tprs_upper,
    color="grey",
    alpha=0.2,
    label=r"$\pm$ 1 std. dev.",
)
title = "Receiver operating characteristic example"
ax.set(
    xlim=[-0.05, 1.05],
    ylim=[-0.05, 1.05],
    title=title,
)
ax.legend(bbox_to_anchor=(0.5,-0.18), loc="upper center")
# plt.savefig(title)
#--------------------------------------------------------
#--------------------------------------------------------

In [ ]:
#--------------------------------------------------------------------------------------
#------ Cross validated Feature Importance Graphs
#--------------------------------------------------------------------------------------
Xc=np.asarray(X)
yc=np.asarray(y)
sgkf = StratifiedGroupKFold(n_splits = 5)
model = XGBClassifier(use_label_encoder=False,
                      eval_metric="auc",
                      objective='binary:logistic',
                      #tree_method='gpu_hist',
                      #gpu_id=0,
                      #predictor='gpu_predictor',
                      scale_pos_weight = scale_pos_weight,
                      n_estimators = 140,
                      learning_rate = 0.34,
                      max_depth = 1,
                      gamma = 0,
                      min_child_weight = 0,
                      subsample = 0.6,
                      colsample_bytree = 0.92,
                      n_jobs=20,
                      alpha=0.17,
                      reg_lambda = 0.86)
output = cross_validate(model,Xc,yc,cv=sgkf, scoring='roc_auc',groups=ids,return_estimator=True)
importances = []
for idx,estimator in enumerate(output['estimator']):
    print("Features sorted by their score for estimator {}:".format(idx))
    feature_importances = estimator.feature_importances_
    importances.append(feature_importances)
plotdata = pd.DataFrame({"Fold 1":importances[0], "Fold 2":importances[1],"Fold 3":importances[2],"Fold 4":importances[3], "Fold 5":importances[4]},index=list(X.columns.values))

plt.rcParams["figure.figsize"] = (4,12)
plotdata = plotdata.sort_values("Fold 1",ascending=False)
plotdata.plot(kind = "barh")
title = "Cross Validated Feature Importance: XGBoost"
plt.title(title,fontsize = 14)

plt.xlabel("Mean Decrease in Impurity",fontsize = 12)
plt.ylabel("Feature",fontsize = 12)
plt.savefig(title, bbox_inches='tight')
#--------------------------------------------------------
#--------------------------------------------------------

In [ ]:
#--------------------------------------------------------------------------------------
#------ Narrow Grid Search
#--------------------------------------------------------------------------------------
X_train = X
y_train = y
params = {
        'n_estimators': [10,20,30,40,70],
        'max_depth': [1],
        'learning_rate': [v/100 for v in range(20,101,10)],
        'min_child_weight':[0],
        'gamma':[0.36],
        'colsample_bytree':[0.93],
        'subsample':[0.6],
        'lambda':[1],
        'alpha':[0]
}
sgkf = StratifiedGroupKFold(n_splits = 5)
model = XGBClassifier(use_label_encoder=False,
                      eval_metric="auc",
                      objective='binary:logistic',
                      tree_method='gpu_hist',
                      gpu_id=0,
                      predictor='gpu_predictor',
                      scale_pos_weight = scale_pos_weight,
                      n_jobs=60)
model = RandomForestClassifier(n_jobs = 60)
tuner = GridSearchCV(model, param_grid=params, verbose=10, cv=sgkf, scoring= 'roc_auc', return_train_score = True)
tuner.fit(X_train, y_train, groups = ids)
print(tuner.best_params_)
#--------------------------------------------------------
#--------------------------------------------------------

In [ ]:
#--------------------------------------------------------------------------------------
#------ Wide Random Halving Search
#--------------------------------------------------------------------------------------

# HalvingRandomSearchCV
X_train = X
y_train = y
params = {
        'max_depth': range(1,30,1),
        'max_features': range(1,25,1),
        'min_samples_split':range(2,31,1),
        'min_samples_leaf':range(1,31,1),
}
sgkf = StratifiedGroupKFold(n_splits = 5)
model = XGBClassifier(use_label_encoder=False,
                      eval_metric="auc",
                      objective='binary:logistic',
                      tree_method='gpu_hist',
                      gpu_id=0,
                      predictor='gpu_predictor',
                      scale_pos_weight = scale_pos_weight,
                      n_jobs=60)
tuner = HalvingRandomSearchCV(model, param_distributions=params, verbose=10, cv=sgkf, scoring= 'roc_auc',
                           resource='n_estimators',
                           min_resources = 5,
                               max_resources=500,
#                             aggressive_elimination = True,
                               random_state=0,
                              factor = 2,
                           return_train_score= True)
tuner.fit(X_train, y_train, groups = ids)
print(tuner.best_params_)
#--------------------------------------------------------
#--------------------------------------------------------

In [ ]:
#--------------------------------------------------------------------------------------
#------ Save Model
#--------------------------------------------------------------------------------------
joblib.dump(tuner, 'xgbgridsearch1.pkl')
print(tuner)
tuner.best_score_
tuner.cv_results_.keys()
#--------------------------------------------------------
#--------------------------------------------------------